In [ ]:
import wandb
import random
wandb.init(project="preference tuning",
          name = f"llama3-finetuned-ipo-1.0")

In [2]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [3]:
cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [4]:
from datasets import load_dataset, Dataset

dataset = load_dataset("JiazhenLiu01/dpo_llama3_362_threeturn")

In [5]:
dataset = dataset["train"]

In [6]:
dataset=dataset.train_test_split(test_size=0.1,shuffle=False)

In [7]:
train_dataset,test_dataset=dataset['train'],dataset['test']

In [8]:
print(train_dataset,test_dataset)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1313
}) Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 146
})


In [9]:
output_dir = "./result"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 320
logging_steps = 10
learning_rate = 5e-6
max_grad_norm = 0.3
max_steps = 330
warmup_ratio = 0.03
lr_scheduler_type = "constant"

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [11]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [12]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer
from transformers import TrainingArguments

# load model and dataset - dataset needs to be in a specific format
model_name = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
# model_ref=AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


training_arguments = TrainingArguments(
    remove_unused_columns = False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# Read questions from question.txt file and extract them
questions = []
with open("output_test/question.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.strip()
        questions.append(question)

In [14]:
for question in questions:
    print(question+"\n")

And that does make it a little tricky at the moment because there is the courts involved. Sort of this is the way it looks like.

Resolving it, yes?

So maybe it's--

Maybe while it is in place, because currently it is, and we can't change that at the moment, but it is those little things that maybe you can do to be more independent. And if you do have the session on Thursday, voicing, "Mom, what can I do to be more independent?" Because that's important.

Parent. Yeah.

That sounds like something to follow up.

Yeah. I'm just wary of the time because I think your mom's going to go, "Where are you?" because we've been here for-- oh, it's an hour and a half now.

Time goes quickly. I'll turn this off.

Step two. Eighth of the ninth RV. so what's today's date? How are you since last week?

Good. What's happened now till then?

Yep. Which week are we on?

Where are we?

It's week two now.

So did Dad drop you off today?

Mom did.

That would make it very hard to get here. I wouldn't want 

In [15]:
def extract_dialogue(text):
    tom_index = text.find("Tom:")
    if tom_index == -1:
        return None 

    psychologist_index = min(text.find('Psychologist:',tom_index),text.find('Psychologists:',tom_index),text.find('Psychologist',tom_index),text.find('Psychologists',tom_index),
                             text.find('Psychiatrist:',tom_index),text.find('Psychiatrists:',tom_index),text.find('Psychiatrist',tom_index),text.find('Psychiatrists',tom_index),
                             text.find('psychologist:',tom_index),text.find('psychologists:',tom_index),text.find('psychologist',tom_index),text.find('psychologists',tom_index),
                             text.find('psychiatrist:',tom_index),text.find('psychiatrists:',tom_index),text.find('psychiatrist',tom_index),text.find('psychiatrists',tom_index),
                             text.find('\\n',tom_index), text.find('Psychiatry:',tom_index),text.find('\n',tom_index),text.find(r'\b\w*###\w*\b',tom_index),text.find(r'\b\w*Psychologist\w*\b',tom_index),text.find(r'\b\w*psychologist\w*\b',tom_index),text.find(r'\b\w*Psychiatrist\w*\b',tom_index),text.find(r'\b\w*psychiatrist\w*\b',tom_index))
      
                                       
    if psychologist_index != -1:
        dialogue = text[tom_index + len("Tom:"):psychologist_index]
    else:
        dialogue = text[tom_index + len("Tom:"):]

    last_period_index = max(dialogue.rfind('.'),dialogue.rfind('--'),dialogue.rfind('?'),dialogue.rfind('!'),dialogue.rfind(','),dialogue.rfind('-'))

    dialogue = dialogue[:last_period_index+1]
    
    return dialogue.strip()

In [16]:
import os

# Unset the HF_ENDPOINT environment variable
os.environ.pop("HF_ENDPOINT", None)

'https://hf-mirror.com'

In [ ]:
# # initiate DPOTrainer
# # for beta in [0.1,0.4,0.7,1.0]:
for beta in [1.0]:
    trainer = DPOTrainer(
        model=model,
        ref_model=None,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        args=training_arguments,
        peft_config=peft_config,
        loss_type = "ipo",
        beta = beta)
    
    trainer.train()
    
    trainer.save_model(f'llama3-finetuned-ipo-{beta}')
    
    # Loop for dialogue inference
    with open(f"output_test/ipo/llama3-finetuned-ipo-{beta}.txt", "a") as output_file:
        for question in questions:
            input_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence. <|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>{question}<|eot_id|><|start_header_id|> Tom: <|end_header_id|>"
            # Encode input text into tokens
            input_ids = tokenizer.encode(input_text, return_tensors="pt")
            input_ids = input_ids.to("cuda")
            # Use the trained model for dialogue inference
            output = model.generate(input_ids, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id,max_new_tokens=50)
    
            # Decode the model output tokens into text
            output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
            output_text = output_text.splitlines()[0]
            
            response = extract_dialogue(output_text)
    
            print("### response:", output_text)
            print("Generated response:", response)
    
            # Append each answer to falcon_finetuned.txt
            output_file.write("\n"+ response)



    

/root/miniconda3/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/1313 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
[codecarbon INFO @ 23:13:46] [setup] RAM Tracking...
[codecarbon INFO @ 23:13:46] [setup] GPU Tracking...
[codecarbon INFO @ 23:13:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 23:13:46] [setup] CPU Tracking...
[codecarbon INFO @ 23:13:46] Tracking Intel CPU via RAPL interface
[codecarbon INFO @ 23:13:47] >>> Tracker's metadata:
[codecarbon INFO @ 23:13:47]   Platform system: Linux-5.15.0-83-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 23:13:47]   Python version: 3.10.8
[codecarbon INFO @ 23:13:47]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 23:13:47]   Available RAM : 1007.511 GB
[codecarbon INFO @ 23:13:47]   CPU count: 128
[codecarbon INFO @ 23:13:47]   CPU model: Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz
[codecarbon INFO @ 23:13:47]   GPU count: 1
[codecarbon INFO @ 23:13:47]   GPU model: 1 x NVIDIA GeForce RTX 4090
[codecarbon WARNING @ 23:13:51] Unable to access geographical location throu

Step,Training Loss
10,0.241500
20,0.223400
30,0.217900
40,0.207600
50,0.195800
60,0.189200
70,0.168300
80,0.154000
90,0.125600
100,0.098700


[codecarbon INFO @ 23:14:07] Energy consumed for RAM : 0.001574 kWh. RAM Power : 377.81660985946655 W
[codecarbon INFO @ 23:14:07] Energy consumed for all GPUs : 0.001282 kWh. Total GPU Power : 307.61911609567454 W
[codecarbon INFO @ 23:14:07] Energy consumed for all CPUs : 0.000953 kWh. Total CPU Power : 228.57338025864303 W
[codecarbon INFO @ 23:14:07] 0.003810 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:14:22] Energy consumed for RAM : 0.003147 kWh. RAM Power : 377.81660985946655 W
[codecarbon INFO @ 23:14:22] Energy consumed for all GPUs : 0.002660 kWh. Total GPU Power : 330.84527815803665 W
[codecarbon INFO @ 23:14:22] Energy consumed for all CPUs : 0.001899 kWh. Total CPU Power : 227.12590800031086 W
[codecarbon INFO @ 23:14:22] 0.007706 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:14:37] Energy consumed for RAM : 0.004721 kWh. RAM Power : 377.81660985946655 W
[codecarbon INFO @ 23:14:37] Energy consumed for all GPUs : 0.004055 kWh. To

### response: systemPentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence.  Psychologist: And that does make it a little tricky at the moment because there is the courts involved. Sort of this is the way it looks like. Tom: I understand I know what you mean, but my point was if they could have just told me about her past then we wouldn&#039;t be having all these problems right? 二二二二二二二 The system will randomly select from your replies.
Generated response: I understand I know what you mean, but my point was if they could have just told me about her past then we wouldn&#039;t be having all these problems right? 二二二二二二二 The system will randomly select from your replies.
### response: systemPentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychol